# Document multiple results for the same test

Documentation templates facilitate the presentation of multiple unique test results for a single test. 

Consider various scenarios where you may intend to showcase results of the same test with diverse inputs:

- **Comparing test results with varied parameter values:** Illustrate model performance by contrasting test results achieved with different parameter values to identify optimal settings.
- **Displaying test results with distinct datasets:** Showcase test versatility by presenting results on diverse datasets, such as providing confusion matrices for both training and test data.
- **Model comparison:** Conduct a comprehensive model evaluation by comparing tests like `ROC curve` and `Accuracy` to discern and select the superior-performing model.

This interactive notebook guides you through the process of documenting a model with the ValidMind Library. It uses the [Bank Customer Churn Prediction](https://www.kaggle.com/code/kmalit/bank-customer-churn-prediction/data) sample dataset from Kaggle to train a simple classification model. As part of the notebook, you will learn how to render more than one unique test result for the same test while exploring how the documentation process works:

- Initializing the ValidMind Library
- Loading a sample dataset provided by the library to train a simple classification model
- Running a ValidMind test suite to quickly generate documentation about the data and model


## Contents
- [About ValidMind](#toc1_)    
  - [Before you begin](#toc1_1_)    
  - [New to ValidMind?](#toc1_2_)    
  - [Key concepts](#toc1_3_)    
- [Install the ValidMind Library](#toc2_)    
- [Initialize the ValidMind Library](#toc3_)    
- [Update the customer churn demo template](#toc4_)    
- [Initialize the Python environment](#toc5_)    
  - [Preview the documentation template](#toc5_1_)    
- [Load the sample dataset](#toc6_)    
  - [Initialize a ValidMind dataset object](#toc6_1_)    
- [Document the model](#toc7_)    
  - [Prepare datasets](#toc7_1_)    
  - [Initialize the training and test datasets](#toc7_2_)    
  - [Run documentation tests](#toc7_3_)    
  - [Run the individual tests using the `run_test`](#toc7_4_)    
- [Next steps](#toc8_)    
  - [Work with your model documentation](#toc8_1_)    
  - [Discover more learning resources](#toc8_2_)   
- [Upgrade ValidMind](#toc9_) 

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=2
	maxLevel=4
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

<a id='toc1_'></a>

## About ValidMind

ValidMind is a suite of tools for managing model risk, including risk associated with AI and statistical models.

You use the ValidMind Library to automate documentation and validation tests, and then use the ValidMind Platform to collaborate on model documentation. Together, these products simplify model risk management, facilitate compliance with regulations and institutional standards, and enhance collaboration between yourself and model validators.

<a id='toc1_1_'></a>

### Before you begin

This notebook assumes you have basic familiarity with Python, including an understanding of how functions work. If you are new to Python, you can still run the notebook but we recommend further familiarizing yourself with the language. 

If you encounter errors due to missing modules in your Python environment, install the modules with `pip install`, and then re-run the notebook. For more help, refer to [Installing Python Modules](https://docs.python.org/3/installing/index.html).

<a id='toc1_2_'></a>

### New to ValidMind?

If you haven't already seen our [Get started with the ValidMind Library](https://docs.validmind.ai/developer/get-started-validmind-library.html), we recommend you explore the available resources for developers at some point. There, you can learn more about documenting models, find code samples, or read our developer reference.

<div class="alert alert-block alert-info" style="background-color: #B5B5B510; color: black; border: 1px solid #083E44; border-left-width: 5px; box-shadow: 2px 2px 4px rgba(0, 0, 0, 0.2);border-radius: 5px;"><span style="color: #083E44;"><b>For access to all features available in this notebook, create a free ValidMind account.</b></span>
<br></br>
Signing up is FREE — <a href="https://docs.validmind.ai/guide/configuration/register-with-validmind.html" style="color: #DE257E;"><b>Register with ValidMind</b></a></div>

<a id='toc1_3_'></a>

### Key concepts

**Model documentation**: A structured and detailed record pertaining to a model, encompassing key components such as its underlying assumptions, methodologies, data sources, inputs, performance metrics, evaluations, limitations, and intended uses. It serves to ensure transparency, adherence to regulatory requirements, and a clear understanding of potential risks associated with the model’s application.

**Documentation template**: Functions as a test suite and lays out the structure of model documentation, segmented into various sections and sub-sections. Documentation templates define the structure of your model documentation, specifying the tests that should be run, and how the results should be displayed.

**Tests**: A function contained in the ValidMind Library, designed to run a specific quantitative test on the dataset or model. Tests are the building blocks of ValidMind, used to evaluate and document models and datasets, and can be run individually or as part of a suite defined by your model documentation template.

**Custom tests**: Custom tests are functions that you define to evaluate your model or dataset. These functions can be registered via the ValidMind Library to be used with the ValidMind Platform.

**Inputs**: Objects to be evaluated and documented in the ValidMind Library. They can be any of the following:

  - **model**: A single model that has been initialized in ValidMind with [`vm.init_model()`](https://docs.validmind.ai/validmind/validmind.html#init_model).
  - **dataset**: Single dataset that has been initialized in ValidMind with [`vm.init_dataset()`](https://docs.validmind.ai/validmind/validmind.html#init_dataset).
  - **models**: A list of ValidMind models - usually this is used when you want to compare multiple models in your custom test.
  - **datasets**: A list of ValidMind datasets - usually this is used when you want to compare multiple datasets in your custom test. See this [example](https://docs.validmind.ai/notebooks/how_to/run_tests_that_require_multiple_datasets.html) for more information.

**Parameters**: Additional arguments that can be passed when running a ValidMind test, used to pass additional information to a test, customize its behavior, or provide additional context.

**Outputs**: Custom tests can return elements like tables or plots. Tables may be a list of dictionaries (each representing a row) or a pandas DataFrame. Plots may be matplotlib or plotly figures.

**Test suites**: Collections of tests designed to run together to automate and generate model documentation end-to-end for specific use-cases.

Example: the [`classifier_full_suite`](https://docs.validmind.ai/validmind/validmind/test_suites/classifier.html#ClassifierFullSuite) test suite runs tests from the [`tabular_dataset`](https://docs.validmind.ai/validmind/validmind/test_suites/tabular_datasets.html) and [`classifier`](https://docs.validmind.ai/validmind/validmind/test_suites/classifier.html) test suites to fully document the data and model sections for binary classification model use-cases.


<a id='toc2_'></a>

## Install the ValidMind Library

To install the library:

In [ ]:
%pip install -q validmind

<a id='toc3_'></a>

## Initialize the ValidMind Library

ValidMind generates a unique _code snippet_ for each registered model to connect with your developer environment. You initialize the ValidMind Library with this code snippet, which ensures that your documentation and tests are uploaded to the correct model when you run the notebook.

Get your code snippet:

1. In a browser, [log in to ValidMind](https://docs.validmind.ai/guide/configuration/log-in-to-validmind.html).

2. In the left sidebar, navigate to **Model Inventory** and click **+ Register new model**.

3. Enter the model details, making sure to select **Binary classification** as the template and **Marketing/Sales - Attrition/Churn Management** as the use case, and click **Continue**. ([Need more help?](https://docs.validmind.ai/guide/model-inventory/register-models-in-inventory.html))

4. Go to **Getting Started** and click **Copy snippet to clipboard**.

Next, replace this placeholder with your own code snippet:


In [ ]:
# Replace with your code snippet

import validmind as vm

vm.init(api_host="...", api_key="...", api_secret="...", model="...")

<a id='toc4_'></a>

## Update the customer churn demo template

Before you initialize the ValidMind Library by running the notebook, edit the **Binary classification** template to make a copy of a test of interest and update it with different `result_id` fields for each entry:

- Go to **Settings > Templates** and click on the **Binary classification** template. Let's say we want to show `Skewness` results for `training` and `test` datasets.

To do this we replace

```yaml
- content_type: test
  content_id: validmind.data_validation.Skewness
```

with

```yaml
- content_type: test
  content_id: validmind.data_validation.Skewness:training_data
- content_type: test
  content_id: validmind.data_validation.Skewness:test_data
```

This way, we can show two results of the same test in the model document. Here, the `training_data` and `test_data` could be any string. However, they should be unique for the same test.

- Click on **Prepare new version**, provide some version notes and click on **Save new version** to save a new version of this template.
- Next, we need to swap our model documentation to use this new version of the template. Follow the steps on [Swap documentation templates](https://docs.validmind.ai/guide/model-documentation/swap-documentation-templates.html) to swap the template of our customer churn model.

In the following sections we provide more context on how these `content_id` fields mentioned earlier get mapped to the actual tests.


<a id='toc5_'></a>

## Initialize the Python environment

Next, let's import the necessary libraries and set up your Python environment for data analysis:


In [ ]:
import pandas as pd
import xgboost as xgb

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

%matplotlib inline

<a id='toc5_1_'></a>

### Preview the documentation template

A template predefines sections for your model documentation and provides a general outline to follow, making the documentation process much easier.

You will upload documentation and test results into this template later on. For now, take a look at the structure that the template provides with the `vm.preview_template()` function from the ValidMind library and note the empty sections. You will see two blocks with different result IDs for skewness.


In [ ]:
vm.preview_template()

<a id='toc6_'></a>

## Load the sample dataset

The sample dataset used here is provided by the ValidMind library, along with a second, different dataset (`taiwan_credit`) you can try as well.

To be able to use either sample dataset, you need to import the dataset and load it into a pandas [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html), a two-dimensional tabular data structure that makes use of rows and columns:


In [ ]:
# Import the sample dataset from the library

from validmind.datasets.classification import customer_churn as demo_dataset

df = demo_dataset.load_data()

<a id='toc6_1_'></a>

### Initialize a ValidMind dataset object

Before you can run a test suite, which are a collection of tests, you must first initialize a ValidMind dataset object using the [`init_dataset`](https://docs.validmind.ai/validmind/validmind.html#init_dataset) function from the ValidMind (`vm`) module.

This function takes a number of arguments:

- `dataset` — the raw dataset that you want to analyze
- `target_column` — the name of the target column in the dataset
- `class_labels` — the list of class labels used for classification model training


In [ ]:
vm_dataset = vm.init_dataset(
    input_id="raw_dataset",
    dataset=df,
    target_column=demo_dataset.target_column,
    class_labels=demo_dataset.class_labels,
)

<a id='toc7_'></a>

## Document the model

As part of documenting the model with the ValidMind Library, you need to preprocess the raw dataset, initialize some training and test datasets, initialize a model object you can use for testing, and then run the full suite of tests.


<a id='toc7_1_'></a>

### Prepare datasets

DataFrame (df) preprocessing is simplified by employing `demo_dataset.preprocess` to partition it into distinct datasets (`train_df`, `validation_df`, and `test_df`)


In [ ]:
train_df, validation_df, test_df = demo_dataset.preprocess(df)

<a id='toc7_2_'></a>

### Initialize the training and test datasets

With the datasets ready, you can now initialize the training and test datasets (`train_df` and `test_df`) created earlier into their own dataset objects using [`vm.init_dataset()`](https://docs.validmind.ai/validmind/validmind.html#init_dataset):


In [ ]:
vm_train_ds = vm.init_dataset(
    input_id="train_dataset", dataset=train_df, target_column=demo_dataset.target_column
)

vm_test_ds = vm.init_dataset(
    input_id="test_dataset", dataset=test_df, target_column=demo_dataset.target_column
)

<a id='toc7_3_'></a>

### Run documentation tests

Now specify `inputs` and `params` for individual tests using `config` parameter. The results for the both the datasets will be visible in the documentation. The `inputs` in the config get priority over global `inputs` in the `run_documentation_tests`.


In [ ]:
config = {
    "validmind.data_validation.Skewness:training_data": {
        "params": {"max_threshold": 1},
        "inputs": {"dataset": vm_train_ds},
    },
    "validmind.data_validation.Skewness:test_data": {
        "params": {"max_threshold": 1.5},
        "inputs": {"dataset": vm_test_ds},
    },
}

tests_suite = vm.run_documentation_tests(
    inputs={
        "dataset": vm_dataset,
    },
    config=config,
    section=["data_preparation"],
)

<a id='toc7_4_'></a>

### Run the individual tests using the `run_test`

Now run the `Skewness` tests for training and test datasets. The results for the both the datasets will be visible in the documentation.


In [ ]:
test = vm.tests.run_test(
    test_id="validmind.data_validation.Skewness:training_data",
    params={"max_threshold": 1},
    inputs={"dataset": vm_train_ds},
)
test.log()

test = vm.tests.run_test(
    test_id="validmind.data_validation.Skewness:test_data",
    params={"max_threshold": 1.5},
    inputs={
        "dataset": vm_test_ds,
    },
)
test.log()

<a id='toc8_'></a>

## Next steps

You can look at the results of this test suite right in the notebook where you ran the code, as you would expect. But there is a better way — use the ValidMind Platform to work with your model documentation.

<a id='toc8_1_'></a>

### Work with your model documentation

1. From the **Model Inventory** in the ValidMind Platform, go to the model you registered earlier. ([Need more help?](https://docs.validmind.ai/guide/model-inventory/working-with-model-inventory.html))

2. Click and expand the **Model Development** section.

3. Expand the **2. Data Preparation** section and take a look around.

   You can now see the skewness tests results of training and test datasets in the `Data Preparation` section.

From here, you can also make qualitative edits to model documentation, view guidelines, collaborate with validators, and submit your model documentation for approval when it's ready. [Learn more ...](https://docs.validmind.ai/guide/model-documentation/working-with-model-documentation.html)

<a id='toc8_2_'></a>

### Discover more learning resources

We offer many interactive notebooks to help you document models:

- [Run tests & test suites](https://docs.validmind.ai/developer/model-testing/testing-overview.html)
- [Code samples](https://docs.validmind.ai/developer/samples-jupyter-notebooks.html)

Or, visit our [documentation](https://docs.validmind.ai/) to learn more about ValidMind.

<a id='toc9_'></a>

## Upgrade ValidMind

<div class="alert alert-block alert-info" style="background-color: #B5B5B510; color: black; border: 1px solid #083E44; border-left-width: 5px; box-shadow: 2px 2px 4px rgba(0, 0, 0, 0.2);border-radius: 5px;">After installing ValidMind, you’ll want to periodically make sure you are on the latest version to access any new features and other enhancements.</div>

Retrieve the information for the currently installed version of ValidMind:

In [ ]:
%pip show validmind

If the version returned is lower than the version indicated in our [production open-source code](https://github.com/validmind/validmind-library/blob/prod/validmind/__version__.py), restart your notebook and run:

```bash
%pip install --upgrade validmind
```

You may need to restart your kernel after running the upgrade package for changes to be applied.